In [1]:
sc

In [4]:
!rm -rf metastore_db/*.lck

from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

In [5]:
from pyspark.mllib.util import MLUtils

!wget https://raw.githubusercontent.com/apache/spark/master/data/mllib/sample_libsvm_data.txt

data = MLUtils.loadLibSVMFile(sc, "sample_libsvm_data.txt").toDF()
data = MLUtils.convertVectorColumnsToML(data)

--2016-09-26 20:31:47--  https://raw.githubusercontent.com/apache/spark/master/data/mllib/sample_libsvm_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.12.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.12.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104736 (102K) [text/plain]
Saving to: ‘sample_libsvm_data.txt.2’

100%[======================================>] 104.736     --.-K/s   in 0,09s   

2016-09-26 20:31:48 (1,10 MB/s) - ‘sample_libsvm_data.txt.2’ saved [104736/104736]



In [12]:
data.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(692,[127,128,129...|  0.0|
|(692,[158,159,160...|  1.0|
|(692,[124,125,126...|  1.0|
|(692,[152,153,154...|  1.0|
|(692,[151,152,153...|  1.0|
+--------------------+-----+
only showing top 5 rows



## Decision Trees

### Classification

In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, IndexToString, VectorIndexer

labelIndexer = StringIndexer().setInputCol("label").setOutputCol("indexedLabel").fit(data)

labelConverter = IndexToString().setInputCol("prediction").setOutputCol("predictedLabel").setLabels(labelIndexer.labels)

featureIndexer = VectorIndexer().setInputCol("features").setOutputCol("indexedFeatures").setMaxCategories(4).fit(data)

dtC = DecisionTreeClassifier().setLabelCol("indexedLabel").setFeaturesCol("indexedFeatures")

pipelineClass = Pipeline().setStages([labelIndexer, featureIndexer, dtC, labelConverter])

trainingData, testData = data.randomSplit([0.7, 0.3])

In [8]:
from pyspark.ml.classification import DecisionTreeClassifier, DecisionTreeClassificationModel

modelClassifier = pipelineClass.fit(trainingData)

treeModel = modelClassifier.stages[2]

predictionsClass = modelClassifier.transform(testData)

In [9]:
modelClassifier.stages

[StringIndexer_45e0a3cbec8d55001833,
 VectorIndexer_49e9bb8409501dddeaec,
 DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4b9dafd963214991d58f) of depth 2 with 5 nodes,
 IndexToString_41da84ef0155ffecb971]

In [10]:
print treeModel.toDebugString

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4b9dafd963214991d58f) of depth 2 with 5 nodes
  If (feature 406 <= 20.0)
   If (feature 99 in {2.0})
    Predict: 0.0
   Else (feature 99 not in {2.0})
    Predict: 1.0
  Else (feature 406 > 20.0)
   Predict: 0.0



In [11]:
predictionsClass.toPandas()[:5]

,features,label,indexedLabel,indexedFeatures,rawPrediction,probability,prediction,predictedLabel
0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 27.0]","[0.0, 1.0]",1.0,0.0
1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 27.0]","[0.0, 1.0]",1.0,0.0
2,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 27.0]","[0.0, 1.0]",1.0,0.0
3,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[37.0, 0.0]","[1.0, 0.0]",0.0,1.0
4,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[37.0, 0.0]","[1.0, 0.0]",0.0,1.0


### Regression

In [14]:
from pyspark.ml.regression import DecisionTreeRegressor, DecisionTreeRegressionModel

dtR = DecisionTreeRegressor().setLabelCol("label").setFeaturesCol("indexedFeatures")

pipelineReg = Pipeline().setStages([featureIndexer, dtR])

In [15]:
modelRegressor = pipelineReg.fit(trainingData)

treeModel = modelRegressor.stages[1]

print treeModel.toDebugString

DecisionTreeRegressionModel (uid=DecisionTreeRegressor_4457a5d93742a969bab8) of depth 2 with 5 nodes
  If (feature 406 <= 20.0)
   If (feature 99 in {0.0})
    Predict: 0.0
   Else (feature 99 not in {0.0})
    Predict: 1.0
  Else (feature 406 > 20.0)
   Predict: 1.0



In [16]:
predictionsReg = modelRegressor.transform(testData)

In [17]:
predictionsReg.toPandas()[:5]

,features,label,indexedFeatures,prediction
0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0
1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0
2,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0
3,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0
4,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1.0


In [1]:
sc

In [2]:
!rm -rf metastore_db/*.lck

from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

In [3]:
from pyspark.mllib.util import MLUtils

data = MLUtils.loadLibSVMFile(sc, "sample_libsvm_data.txt").toDF()
data = MLUtils.convertVectorColumnsToML(data)

In [4]:
data.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(692,[127,128,129...|  0.0|
|(692,[158,159,160...|  1.0|
|(692,[124,125,126...|  1.0|
|(692,[152,153,154...|  1.0|
|(692,[151,152,153...|  1.0|
+--------------------+-----+
only showing top 5 rows



## Random Forests

### Classification

In [6]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, IndexToString, VectorIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import RandomForestClassificationModel

labelIndexer = StringIndexer().setInputCol("label") \
                                .setOutputCol("indexedLabel").fit(data)

labelConverter = IndexToString().setInputCol("prediction") \
                                .setOutputCol("predictedLabel") \
                                .setLabels(labelIndexer.labels)

featureIndexer = VectorIndexer().setInputCol("features") \
                                .setOutputCol("indexedFeatures") \
                                .setMaxCategories(4).fit(data)

rfC = RandomForestClassifier().setLabelCol("indexedLabel") \
                                .setFeaturesCol("indexedFeatures") \
                                .setNumTrees(3)
        
trainingData, testData = data.randomSplit([0.7, 0.3])

In [7]:
pipelineRFC = Pipeline().setStages([labelIndexer, featureIndexer, rfC, labelConverter])

modelRFC = pipelineRFC.fit(trainingData)

predictionsRFC = modelRFC.transform(testData)

In [8]:
predictionsRFC.select("predictedLabel", "label", "features").show(5)

+--------------+-----+--------------------+
|predictedLabel|label|            features|
+--------------+-----+--------------------+
|           1.0|  1.0|(692,[97,98,99,12...|
|           0.0|  0.0|(692,[122,123,148...|
|           1.0|  1.0|(692,[123,124,125...|
|           1.0|  1.0|(692,[123,124,125...|
|           0.0|  0.0|(692,[124,125,126...|
+--------------+-----+--------------------+
only showing top 5 rows



In [9]:
rfModelC = modelRFC.stages[2]
rfModelC.featureImportances

SparseVector(692, {183: 0.0326, 455: 0.2952, 463: 0.2636, 490: 0.3333, 517: 0.0371, 540: 0.0381})

In [10]:
print rfModelC.toDebugString

RandomForestClassificationModel (uid=rfc_d5d6b33eed79) with 3 trees
  Tree 0 (weight 1.0):
    If (feature 463 <= 0.0)
     If (feature 183 <= 0.0)
      Predict: 0.0
     Else (feature 183 > 0.0)
      If (feature 517 <= 116.0)
       Predict: 1.0
      Else (feature 517 > 116.0)
       Predict: 0.0
    Else (feature 463 > 0.0)
     Predict: 0.0
  Tree 1 (weight 1.0):
    If (feature 490 <= 31.0)
     Predict: 1.0
    Else (feature 490 > 31.0)
     Predict: 0.0
  Tree 2 (weight 1.0):
    If (feature 455 <= 23.0)
     If (feature 540 <= 65.0)
      Predict: 0.0
     Else (feature 540 > 65.0)
      Predict: 1.0
    Else (feature 455 > 23.0)
     Predict: 1.0



### Regression

In [11]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import RandomForestRegressionModel

rfR = RandomForestRegressor().setLabelCol("label").setFeaturesCol("indexedFeatures")

pipelineRFR = Pipeline().setStages([featureIndexer, rfR])

modelRFR = pipelineRFR.fit(trainingData)

predictionsRFR = modelRFR.transform(testData)

In [12]:
predictionsRFR = modelRFR.transform(testData)

predictionsRFR.select("prediction", "label", "features").show(5)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.9|  1.0|(692,[97,98,99,12...|
|      0.05|  0.0|(692,[122,123,148...|
|       1.0|  1.0|(692,[123,124,125...|
|      0.95|  1.0|(692,[123,124,125...|
|       0.0|  0.0|(692,[124,125,126...|
+----------+-----+--------------------+
only showing top 5 rows



In [1]:
sc

In [3]:
!rm -rf metastore_db/*.lck

from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

In [4]:
from pyspark.mllib.util import MLUtils

data = MLUtils.loadLibSVMFile(sc, "sample_libsvm_data.txt").toDF()
data = MLUtils.convertVectorColumnsToML(data)

In [5]:
data.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(692,[127,128,129...|  0.0|
|(692,[158,159,160...|  1.0|
|(692,[124,125,126...|  1.0|
|(692,[152,153,154...|  1.0|
|(692,[151,152,153...|  1.0|
+--------------------+-----+
only showing top 5 rows



## Gradient Boosted Trees

### Classification

In [11]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, IndexToString, VectorIndexer
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import GBTClassificationModel

labelIndexer = StringIndexer().setInputCol("label") \
                                .setOutputCol("indexedLabel").fit(data)

labelConverter = IndexToString().setInputCol("prediction") \
                                .setOutputCol("predictedLabel") \
                                .setLabels(labelIndexer.labels)

featureIndexer = VectorIndexer().setInputCol("features") \
                                .setOutputCol("indexedFeatures") \
                                .setMaxCategories(4).fit(data)

gbtC = GBTClassifier().setLabelCol("indexedLabel").setFeaturesCol("indexedFeatures").setMaxIter(10)

pipelineGBTC = Pipeline().setStages([labelIndexer, featureIndexer, gbtC, labelConverter])

trainingData, testData = data.randomSplit([0.7, 0.3])

In [12]:
modelGBTC = pipelineGBTC.fit(trainingData)

predictionsGBTC = modelGBTC.transform(testData)

predictionsGBTC.select("predictedLabel", "label", "features").show(3)

+--------------+-----+--------------------+
|predictedLabel|label|            features|
+--------------+-----+--------------------+
|           0.0|  0.0|(692,[122,123,148...|
|           1.0|  1.0|(692,[123,124,125...|
|           0.0|  0.0|(692,[124,125,126...|
+--------------+-----+--------------------+
only showing top 3 rows



In [13]:
gbtModelC = modelGBTC.stages[2]

print gbtModelC.toDebugString

GBTClassificationModel (uid=GBTClassifier_452db09d245c6ad64f92) with 10 trees
  Tree 0 (weight 1.0):
    If (feature 406 <= 72.0)
     If (feature 99 in {2.0})
      Predict: -1.0
     Else (feature 99 not in {2.0})
      Predict: 1.0
    Else (feature 406 > 72.0)
     Predict: -1.0
  Tree 1 (weight 0.1):
    If (feature 406 <= 72.0)
     If (feature 435 <= 0.0)
      If (feature 577 <= 231.0)
       If (feature 123 <= 66.0)
        If (feature 153 <= 3.0)
         Predict: 0.4768116880884702
        Else (feature 153 > 3.0)
         Predict: 0.4768116880884703
       Else (feature 123 > 66.0)
        Predict: 0.4768116880884703
      Else (feature 577 > 231.0)
       Predict: 0.47681168808847035
     Else (feature 435 > 0.0)
      Predict: -0.4768116880884694
    Else (feature 406 > 72.0)
     If (feature 207 <= 140.0)
      Predict: -0.47681168808847024
     Else (feature 207 > 140.0)
      Predict: -0.4768116880884712
  Tree 2 (weight 0.1):
    If (feature 406 <= 72.0)
     If (feat

### Regression

In [15]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import GBTRegressionModel

gbtR = GBTRegressor().setLabelCol("label").setFeaturesCol("indexedFeatures").setMaxIter(10)

pipelineGBTR = Pipeline().setStages([featureIndexer, gbtR])

modelGBTR = pipelineGBTR.fit(trainingData)

In [16]:
predictionsGBTR = modelGBTR.transform(testData)
predictionsGBTR.show(5)

+--------------------+-----+--------------------+----------+
|            features|label|     indexedFeatures|prediction|
+--------------------+-----+--------------------+----------+
|(692,[122,123,148...|  0.0|(692,[122,123,148...|       0.0|
|(692,[123,124,125...|  1.0|(692,[123,124,125...|       1.0|
|(692,[124,125,126...|  0.0|(692,[124,125,126...|       0.0|
|(692,[124,125,126...|  0.0|(692,[124,125,126...|       0.0|
|(692,[124,125,126...|  1.0|(692,[124,125,126...|       1.0|
+--------------------+-----+--------------------+----------+
only showing top 5 rows



In [1]:
sc

In [2]:
from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

In [3]:
from pyspark.mllib.util import MLUtils

data = MLUtils.loadLibSVMFile(sc, "sample_libsvm_data.txt").toDF()
data = MLUtils.convertVectorColumnsToML(data)

In [4]:
data.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(692,[127,128,129...|  0.0|
|(692,[158,159,160...|  1.0|
|(692,[124,125,126...|  1.0|
|(692,[152,153,154...|  1.0|
|(692,[151,152,153...|  1.0|
+--------------------+-----+
only showing top 5 rows



## Linear Methods

### Logistic Regression

In [7]:
from pyspark.ml.classification import LogisticRegression

trainingData, testData = data.randomSplit([0.7, 0.3])

logr = LogisticRegression().setMaxIter(10).setRegParam(0.3).setElasticNetParam(0.8)

logrModel = logr.fit(trainingData)

print "Weights: %s Intercept: %s" % (logrModel.coefficients, logrModel.intercept)

Weights: (692,[235,243,244,262,263,271,272,290,300,328,345,350,351,373,378,379,400,401,405,406,407,428,429,433,434,455,456,461,462,468,483,484,489,490,496,511,512,517,539,540,568],[-0.000366498963754,-1.42795225616e-05,-9.44060912441e-07,-0.000205590003336,-0.000640271226841,-0.00015049613407,-4.82707085543e-05,-0.000149047540375,-0.00024110937152,-3.31762123523e-05,-8.66627952077e-06,9.14036907531e-05,7.91821174721e-05,-0.000217747702831,0.000506741491092,8.12357667511e-05,-3.70834836804e-05,-0.000248022863756,0.000180277568249,0.000670911954862,0.000199036991568,-0.000248992981369,-4.4187919864e-05,0.000534981568858,0.000602296780737,-2.46888169952e-05,-0.000226660517233,0.000178721956572,0.000526650147849,-2.65336833343e-05,-0.000229259905073,-4.95566431909e-05,0.000170380977959,6.28404067389e-05,-0.000148349494778,-0.000431896225065,-0.000217977284272,6.88366318543e-05,-0.000393900871055,-0.000372365159358,-0.000180633333959]) Intercept: 0.311873962216


In [15]:
trainingSummaryLR = logrModel.summary

In [13]:
trainingSummaryLR.roc.show()

+---+--------------------+
|FPR|                 TPR|
+---+--------------------+
|0.0|                 0.0|
|0.0|0.023809523809523808|
|0.0|0.047619047619047616|
|0.0| 0.07142857142857142|
|0.0| 0.09523809523809523|
|0.0| 0.11904761904761904|
|0.0| 0.14285714285714285|
|0.0| 0.16666666666666666|
|0.0| 0.19047619047619047|
|0.0| 0.21428571428571427|
|0.0| 0.23809523809523808|
|0.0|  0.2619047619047619|
|0.0|  0.2857142857142857|
|0.0| 0.30952380952380953|
|0.0|  0.3333333333333333|
|0.0| 0.35714285714285715|
|0.0| 0.38095238095238093|
|0.0| 0.40476190476190477|
|0.0| 0.42857142857142855|
|0.0|  0.4523809523809524|
+---+--------------------+
only showing top 20 rows



In [14]:
trainingSummaryLR.areaUnderROC

1.0

### Linear Regression

In [16]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression().setMaxIter(10).setRegParam(0.3).setElasticNetParam(0.8)

lrModel = lr.fit(trainingData)

print "Weights: %s Intercept: %s" % (lrModel.coefficients, lrModel.intercept)

Weights: (692,[235,262,263,271,272,290,300,323,328,345,350,351,373,377,378,379,400,401,405,406,407,428,429,433,434,435,455,456,461,462,468,483,484,489,490,496,511,512,517,539,540,568],[-2.09476863284e-05,-1.80379795203e-05,-7.76408069301e-05,-8.62510909972e-06,-8.8863598712e-06,-1.63638781455e-05,-2.32113528661e-05,2.07824914011e-05,-6.63419413635e-06,-1.31267310255e-06,9.73660810406e-05,8.67848065341e-05,-2.0818468715e-05,1.69492814487e-05,0.000127924916667,8.73625991595e-05,-8.12877145313e-06,-2.44097080685e-05,0.000105190944195,0.000193955183142,0.000117112902109,-2.38297092758e-05,-8.36554299431e-06,0.000131433521016,0.000184741250333,2.22045575455e-05,-5.96199550215e-06,-2.17969886225e-05,0.000103310686136,0.000129828235781,-5.20120442642e-06,-2.13971309011e-05,-9.62497812109e-06,9.76958909363e-05,8.09841251226e-05,-9.43615158764e-06,-7.2861938126e-05,-2.08047656197e-05,8.45794820099e-05,-6.40729390395e-05,-2.51503732101e-05,-1.55360772201e-05]) Intercept: 0.40386219884


In [17]:
trainingSummaryLLS = lrModel.summary

print "numIterations: %s" % trainingSummaryLLS.totalIterations

numIterations: 11


In [18]:
trainingSummaryLLS.residuals.show(5)

+--------------------+
|           residuals|
+--------------------+
|-0.28172778090489775|
|  0.5072260030209965|
| 0.31386479014178303|
| 0.20612375835211627|
|  0.2894883399407576|
+--------------------+
only showing top 5 rows



In [1]:
sc

In [4]:
!rm -rf metastore_db/*.lck

from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

In [5]:
from pyspark.mllib.util import MLUtils

data = MLUtils.loadLibSVMFile(sc, "sample_libsvm_data.txt").toDF()
data = MLUtils.convertVectorColumnsToML(data)

In [6]:
data.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(692,[127,128,129...|  0.0|
|(692,[158,159,160...|  1.0|
|(692,[124,125,126...|  1.0|
|(692,[152,153,154...|  1.0|
|(692,[151,152,153...|  1.0|
+--------------------+-----+
only showing top 5 rows



## Evaluation

### Binary Classification

In [17]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import BinaryLogisticRegressionSummary

trainingData, testData = data.randomSplit([0.7, 0.3])

logr = LogisticRegression().setMaxIter(10).setRegParam(0.3).setElasticNetParam(0.8)

logrModel = logr.fit(trainingData)

print "Weights: %s Intercept: %s" % (logrModel.coefficients, logrModel.intercept)

Weights: (692,[235,243,244,262,263,271,272,290,300,301,323,328,329,350,351,358,378,379,385,386,401,405,406,407,413,427,428,429,433,434,435,455,456,461,462,469,483,484,489,490,496,497,511,512,517,539,540,568],[-1.08704076463e-05,-0.000225035191295,-1.42785721134e-06,-2.67838954428e-06,-0.000260749167911,-0.000108151886583,-0.000222047179992,-0.000184684690252,-0.000368205319861,-8.45317828463e-05,8.41700764597e-05,-8.99150674055e-06,-2.6018144065e-06,0.000106542521448,0.000214847611674,-8.02001292779e-06,0.000254341232325,0.000218251840905,-3.96099160439e-05,-2.89412079021e-05,-4.45357563079e-06,5.02740058996e-05,0.00084364056682,0.000260914756075,-2.13472989153e-05,-1.2101806874e-06,-0.000135806709886,-9.35571716832e-06,0.000231569631081,0.000943064886038,7.40416456292e-05,-1.3984003133e-05,-0.000254944102754,0.000131412325807,0.000884111338263,-9.88044455622e-05,-8.10057764206e-05,-0.000193685045759,0.00011911644026,0.000142456462059,-0.000123316509266,-8.01517008313e-05,-0.0003900993

In [12]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

predictionsLogR = logrModel.transform(testData)

evaluator = BinaryClassificationEvaluator().setLabelCol("label") \
                            .setRawPredictionCol("rawPrediction") \
                            .setMetricName("areaUnderROC")

roc = evaluator.evaluate(predictionsLogR)
print roc

1.0


### Multiclass Classification

In [18]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, IndexToString, VectorIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import RandomForestClassificationModel

labelIndexer = StringIndexer().setInputCol("label") \
                                .setOutputCol("indexedLabel").fit(data)

labelConverter = IndexToString().setInputCol("prediction") \
                                .setOutputCol("predictedLabel") \
                                .setLabels(labelIndexer.labels)

featureIndexer = VectorIndexer().setInputCol("features") \
                                .setOutputCol("indexedFeatures") \
                                .setMaxCategories(4).fit(data)

rfC = RandomForestClassifier().setLabelCol("indexedLabel") \
                                .setFeaturesCol("indexedFeatures") \
                                .setNumTrees(3)
        
pipelineRFC = Pipeline().setStages([labelIndexer, featureIndexer, rfC, labelConverter])

modelRFC = pipelineRFC.fit(trainingData)

predictionsRFC = modelRFC.transform(testData)

In [19]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator().setLabelCol("indexedLabel") \
                                        .setPredictionCol("prediction") \
                                        .setMetricName("accuracy")

accuracy = evaluator.evaluate(predictionsRFC)

print "Test Error = %s" % (1.0 - accuracy)

Test Error = 0.0606060606061


### Regression

In [20]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import RandomForestRegressionModel

rfR = RandomForestRegressor().setLabelCol("label").setFeaturesCol("indexedFeatures")

pipelineRFR = Pipeline().setStages([featureIndexer, rfR])

modelRFR = pipelineRFR.fit(trainingData)

predictionsRFR = modelRFR.transform(testData)

In [21]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator().setLabelCol("label") \
                                .setPredictionCol("prediction") \
                                .setMetricName("rmse")

rmse = evaluator.evaluate(predictionsRFR)

print "Root Mean Squared Error (RMSE) = %s" % rmse

Root Mean Squared Error (RMSE) = 0.138169855942


### Logistic Regression

In [23]:
from pyspark.ml.classification import LogisticRegression

logr = LogisticRegression().setMaxIter(10).setRegParam(0.3).setElasticNetParam(0.8)

logrModel = logr.fit(trainingData)

print "Weights: %s Intercept: %s" % (logrModel.coefficients, logrModel.intercept)

Weights: (692,[235,243,244,262,263,271,272,290,300,301,323,328,329,350,351,358,378,379,385,386,401,405,406,407,413,427,428,429,433,434,435,455,456,461,462,469,483,484,489,490,496,497,511,512,517,539,540,568],[-1.08704076463e-05,-0.000225035191295,-1.42785721134e-06,-2.67838954428e-06,-0.000260749167911,-0.000108151886583,-0.000222047179992,-0.000184684690252,-0.000368205319861,-8.45317828463e-05,8.41700764597e-05,-8.99150674055e-06,-2.6018144065e-06,0.000106542521448,0.000214847611674,-8.02001292779e-06,0.000254341232325,0.000218251840905,-3.96099160439e-05,-2.89412079021e-05,-4.45357563079e-06,5.02740058996e-05,0.00084364056682,0.000260914756075,-2.13472989153e-05,-1.2101806874e-06,-0.000135806709886,-9.35571716832e-06,0.000231569631081,0.000943064886038,7.40416456292e-05,-1.3984003133e-05,-0.000254944102754,0.000131412325807,0.000884111338263,-9.88044455622e-05,-8.10057764206e-05,-0.000193685045759,0.00011911644026,0.000142456462059,-0.000123316509266,-8.01517008313e-05,-0.0003900993

In [24]:
trainingSummaryLR = logrModel.summary
trainingSummaryLR.areaUnderROC

1.0

In [25]:
fMeasure = trainingSummaryLR.fMeasureByThreshold

fMeasure.show(3)

+------------------+-------------------+
|         threshold|          F-Measure|
+------------------+-------------------+
|0.8165428632958064|               0.05|
|0.8162694676199894|0.09756097560975609|
|0.8162646945004736|0.14285714285714288|
+------------------+-------------------+
only showing top 3 rows



In [26]:
from pyspark.sql import functions as F

maxFMeasure = fMeasure.agg({"F-Measure": "max"}).head()[0]
print maxFMeasure
maxFMeasure = fMeasure.agg(F.max(F.col("F-Measure"))).head()[0]
print maxFMeasure

bestThreshold = fMeasure.where(F.col("F-Measure") == maxFMeasure).select("threshold").head()[0]
print bestThreshold

1.0
1.0
0.613903946897


In [27]:
trainingSummaryLR.pr.show(3)
trainingSummaryLR.precisionByThreshold.show(3)

+-------------------+---------+
|             recall|precision|
+-------------------+---------+
|                0.0|      1.0|
|0.02564102564102564|      1.0|
|0.05128205128205128|      1.0|
+-------------------+---------+
only showing top 3 rows

+------------------+---------+
|         threshold|precision|
+------------------+---------+
|0.8165428632958064|      1.0|
|0.8162694676199894|      1.0|
|0.8162646945004736|      1.0|
+------------------+---------+
only showing top 3 rows



In [28]:
trainingSummaryLR.recallByThreshold.show(3)
trainingSummaryLR.roc.show(3)

+------------------+-------------------+
|         threshold|             recall|
+------------------+-------------------+
|0.8165428632958064|0.02564102564102564|
|0.8162694676199894|0.05128205128205128|
|0.8162646945004736|0.07692307692307693|
+------------------+-------------------+
only showing top 3 rows

+---+-------------------+
|FPR|                TPR|
+---+-------------------+
|0.0|                0.0|
|0.0|0.02564102564102564|
|0.0|0.05128205128205128|
+---+-------------------+
only showing top 3 rows



### Linear Regression

In [30]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression().setMaxIter(10).setRegParam(0.3).setElasticNetParam(0.8)

lrModel = lr.fit(trainingData)

print "Weights: %s Intercept: %s" % (lrModel.coefficients, lrModel.intercept)

Weights: (692,[351,378,379,406,407,433,434,462,490,517,540],[0.000133767304687,0.00018419522405,0.000135554811074,0.000275780406915,0.000159724075215,0.000144485199988,0.000306915977164,0.000288535540884,0.000120301061362,0.000123576266478,-0.00011823301358]) Intercept: 0.34681682126


In [31]:
trainingSummaryLLS = lrModel.summary

print trainingSummaryLLS.explainedVariance

print trainingSummaryLLS.meanAbsoluteError

print trainingSummaryLLS.meanSquaredError

print trainingSummaryLLS.r2

0.0506615309323
0.271181060072
0.0785812855913
0.676967590642


In [32]:
trainingSummaryLLS.residuals.show(3)

print trainingSummaryLLS.rootMeanSquaredError

+-------------------+
|          residuals|
+-------------------+
|0.24739337680570395|
|-0.3169038688245795|
| 0.5361913707012146|
+-------------------+
only showing top 3 rows

0.280323537348


In [1]:
sc

In [4]:
!rm -rf metastore_db/*.lck

from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

## Saving and Loading Models

In [5]:
from pyspark.ml.linalg import Vectors

df = sqlc.createDataFrame([(1.0, Vectors.dense(1.0, 2.0, 3.0)),
                           (1.0, Vectors.dense(2.0, 3.0, 4.0)),
                           (0.0, Vectors.dense(-1.0, 1.0, 2.0)),
                           (0.0, Vectors.dense(-2.0, 3.0, 5.0))]).toDF("label", "features")

In [7]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression()

lrModel = lr.fit(df)
lrModel.transform(df).show()

+-----+--------------+--------------------+--------------------+----------+
|label|      features|       rawPrediction|         probability|prediction|
+-----+--------------+--------------------+--------------------+----------+
|  1.0| [1.0,2.0,3.0]|[-18.070405604445...|[1.41945802370848...|       1.0|
|  1.0| [2.0,3.0,4.0]|[-38.987081234651...|[1.16983808020729...|       1.0|
|  0.0|[-1.0,1.0,2.0]|[19.2085506510254...|[0.99999999545187...|       0.0|
|  0.0|[-2.0,3.0,5.0]|[29.1902958840818...|[0.99999999999978...|       0.0|
+-----+--------------+--------------------+--------------------+----------+



In [8]:
lrModel.save("lrModel.parquet")

In [10]:
from pyspark.ml.classification import LogisticRegressionModel

sameModel = LogisticRegressionModel.load("lrModel.parquet")
sameModel.transform(df).show()

+-----+--------------+--------------------+--------------------+----------+
|label|      features|       rawPrediction|         probability|prediction|
+-----+--------------+--------------------+--------------------+----------+
|  1.0| [1.0,2.0,3.0]|[-18.070405604445...|[1.41945802370848...|       1.0|
|  1.0| [2.0,3.0,4.0]|[-38.987081234651...|[1.16983808020729...|       1.0|
|  0.0|[-1.0,1.0,2.0]|[19.2085506510254...|[0.99999999545187...|       0.0|
|  0.0|[-2.0,3.0,5.0]|[29.1902958840818...|[0.99999999999978...|       0.0|
+-----+--------------+--------------------+--------------------+----------+



In [13]:
!cat lrModel.parquet/metadata/part-00000 

{"class":"org.apache.spark.ml.classification.LogisticRegressionModel","timestamp":1475089043445,"sparkVersion":"2.0.0","uid":"LogisticRegression_43d69252fb4e6810e320","paramMap":{"regParam":0.0,"tol":1.0E-6,"fitIntercept":true,"maxIter":100,"standardization":true,"elasticNetParam":0.0,"probabilityCol":"probability","rawPredictionCol":"rawPrediction","featuresCol":"features","labelCol":"label","predictionCol":"prediction","threshold":0.5}}
